# Decision Trees

**Decision trees** are popular machine læming algorithm that can be used for **clasification** and **regression tasks**.\
The tree itself is a model in decision trees and we would like to estimate an **optimal tree structure** from the given training data.
* The internal nodes contains conditions on features. Depending on the outcome of the comparison, we take an appropriate path in the
tree. The process is repeated until we reach a leaf node.
* In case Of classification, leaf nodes contain label and in case Of regression, the prediction is obtain by taking sample mean Of labels Of the
subset of training examples present in that leaf node.

In this colab, we will implement decision tree for classification with ID3 algorithm.


## Import Libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
eps = np.finfo(float).eps
eps

2.220446049250313e-16

Here `eps` is the smallest representable number. At times we get `Log(0)` or 0 in the denominator, in order to avoid that we are going to use this.

## Load the Dataset

In [3]:
df = pd.DataFrame([['overcast', 'hot', 'high', 'FALSE', 'yes'],
['overcast', 'cool', 'normal', 'TRUE', 'yes'],
['overcast', 'mild', 'high', 'TRUE', 'yes'],
['overcast', 'hot', 'normal', 'FALSE', 'yes'],
['rainy', 'mild', 'high', 'FALSE', 'yes'],
['rainy', 'cool', 'normal', 'FALSE', 'yes'],
['rainy', 'cool', 'normal', 'TRUE', 'no'],
['rainy', 'mild', 'normal', 'FALSE', 'yes'],
['rainy', 'mild', 'high', 'TRUE', 'no'],
['sunny', 'hot', 'high', 'FALSE', 'no'],
['sunny', 'hot', 'high', 'TRUE', 'no'],
['sunny', 'mild', 'high', 'FALSE', 'no'],
['sunny', 'cool', 'normal', 'FALSE', 'yes'],
['sunny', 'mild', 'normal', 'TRUE', 'yes']], columns=['outlook','temp','humidity','windy','play'])

In [6]:
df.shape

(14, 5)

In [8]:
df.keys() # returns column names 

Index(['outlook', 'temp', 'humidity', 'windy', 'play'], dtype='object')

In [24]:
df.loc[df['play'] == 'yes']

,outlook,temp,humidity,windy,play
0,overcast,hot,high,FALSE,yes
1,overcast,cool,normal,TRUE,yes
2,overcast,mild,high,TRUE,yes
3,overcast,hot,normal,FALSE,yes
4,rainy,mild,high,FALSE,yes
5,rainy,cool,normal,FALSE,yes
7,rainy,mild,normal,FALSE,yes
12,sunny,cool,normal,FALSE,yes
13,sunny,mild,normal,TRUE,yes


In [18]:
target = df.keys()[-1]
target

'play'

In [20]:
df.keys()[:-1]

Index(['outlook', 'temp', 'humidity', 'windy'], dtype='object')

In [22]:
df[target].unique()

array(['yes', 'no'], dtype=object)

In [26]:
df[target].value_counts()['yes']

9

## Calculate Entropy of the whole dataset

In [30]:
def find_entropy_whole(df):
  target = df.keys()[-1]

  unique_val = df[target].unique()

  overall_entropy = 0

  for value in unique_val:
    p = df[target].value_counts()[value]/ len(df[target])
    overall_entropy += -p * np.log2(p)
  return overall_entropy


In [31]:
find_entropy_whole(df)

0.9402859586706309

## Calculate Entopy of attribute


In [32]:
def find_entropy_of_attribute(df, attribute):

  target = df.keys()[-1]
  
  values_in_target = df[target].unique()
  values_in_attribute = df[attribute].unique()
  
  entropy_attribute = 0

  for val_in_att in values_in_attribute:

    overall_entropy = 0

    for val_in_target in values_in_target:
      num = len(df[attribute][df[attribute] == val_in_att][df[target] == val_in_target])
      den = len(df[attribute][df[attribute] == val_in_att])
      p = num/(den + eps)

      overall_entropy +=  - p * np.log2(p + eps)

    p2 = den/len(df[target])

    entropy_attribute += -p2*overall_entropy

  return abs(entropy_attribute)




In [34]:
for i in df.keys()[:-1]:
  print(i,find_entropy_of_attribute(df,i))

outlook 0.6935361388961914
temp 0.9110633930116756
humidity 0.7884504573082889
windy 0.892158928262361


## Finding Best attribute


In [41]:
def find_best_attribute(df):

  all_attributes = df.keys()[:-1]
  target = df.keys()[-1]

  IG = []

  for i in all_attributes:
    ig = find_entropy_whole(df) - find_entropy_of_attribute(df,i)
    IG.append(ig)


  index_of_att_with_max_gain = np.argmax(IG)

  best_attribute = all_attributes[index_of_att_with_max_gain]


  return best_attribute

In [38]:
find_best_attribute(df)

outlook


In [43]:
def buildTree(df, tree = None):

  #last column in dataframe
  target = df.keys()[-1]

  #Here we build our desicion tree

  #Get attribute with maximum information gain

  node = find_best_attribute(df)

  attValue = np.unique(df[node])

  # Create an empty dictionary to create tree

  if tree is None:
    tree = {}
    tree[node] = {}

  # We make loop to construct a tree by calling this function recursively
  # In this we check if the subset is pure and stops if it is pure
  for value in attValue:

    subtable = df[df[node] == value].reset_index(drop = True)
    clValue, counts = np.unique(subtable['play'], return_counts = True)

    if len(counts) == 1:
      tree[node][value] = clValue[0]

    else:
      tree[node][value] = buildTree(subtable)


  return tree

buildTree(df )

{'outlook': {'overcast': 'yes',
  'rainy': {'windy': {'FALSE': 'yes', 'TRUE': 'no'}},
  'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}

ID3 in its pure form performs no backtracking in its search. Once it selects an attribute to test at a particular level in the tree, it never backtracks
to reconsider this choice. Therefore, it is susceptible to the usual risks Of hill-climbing search without backtracking: converging to locally
optimal solutions that are not globally optimal
